In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import nltk 
import spacy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import os 
import sys 

In [2]:
path = "D:/InterviewAutomation/data"

In [3]:
excel_files = [file for file in os.listdir(path) if file.endswith(('.xls', '.xlsx'))]

In [4]:
for file in excel_files:
    df = pd.read_excel(path+'/'+file)
    print(df.columns)

Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript'

In [5]:
# # Changing the Columns of the 4th dataset as it does have the Wrong Columns
# df = pd.read_excel(excel_files[3])
# df.columns = ['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
#        'Reason for decision', 'Job Description']
# df.to_excel(path+'/'+'dataset4.xlsx',index=False)

In [6]:
# for file in excel_files:
#     df = pd.read_excel(path+'/'+file)
#     print(df.columns)

In [7]:
# Code for Merging all the datasets and creating the full dataset for traing and testing
dataframes = []

for file in excel_files:
    file_path = os.path.join(path, file)
    df = pd.read_excel(file_path)
    dataframes.append(df)

# Combine all dataframes into a single dataframe
data = pd.concat(dataframes, ignore_index=True)

In [8]:
data.shape

(3250, 8)

In [9]:
data.head()

,ID,Name,Role,Transcript,Resume,decision,Reason for decision,Job Description
0,brenbr359,Brent Brown,Product Manager,Product Manager Interview Transcript\n\nInterv...,Here's a sample resume for Brent Brown applyin...,select,Experience,We are looking for a skilled Product Manager w...
1,jameay305,James Ayala,Software Engineer,Software Engineer Interview Transcript\n\nInte...,Here's a sample resume for James Ayala applyin...,select,Experience,We are looking for a skilled Software Engineer...
2,scotri565,Scott Rivera,Data Engineer,Here is a simulated interview for Scott Rivera...,Here's a sample resume for Scott Rivera applyi...,reject,Experience,We are looking for a skilled Data Engineer wit...
3,emilke232,Emily Kelly,UI Engineer,Interview Transcript: Emily Kelly for UI Engin...,Here's a sample resume for Emily Kelly:\n\nEmi...,select,Experience,We are looking for a skilled UI Engineer with ...
4,ashlra638,Ashley Ray,Data Scientist,Data Scientist Interview Transcript\n\nCompany...,Here's a sample resume for Ashley Ray applying...,reject,Cultural Fit,We are looking for a skilled Data Scientist wi...


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3250 entries, 0 to 3249
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   3250 non-null   object
 1   Name                 3250 non-null   object
 2   Role                 3250 non-null   object
 3   Transcript           3250 non-null   object
 4   Resume               3250 non-null   object
 5   decision             3250 non-null   object
 6   Reason for decision  3250 non-null   object
 7   Job Description      3250 non-null   object
dtypes: object(8)
memory usage: 203.2+ KB


In [11]:
for col in data.columns:
    data[col] = data[col].apply(lambda x: x.lower())

In [12]:
data['decision'].unique()

array(['select', 'reject', 'rejected', 'selected'], dtype=object)

In [13]:
def process_decision(text):
    if text in ['select','selected']:
        return 'select'
    else :
        return 'reject'

In [14]:
data['decision'] = data['decision'].apply(process_decision )

In [15]:
data['Role'].unique()

array(['product manager', 'software engineer', 'data engineer',
       'ui engineer', 'data scientist', 'data analyst', 'ui designer',
       'digital marketing specialist', 'cybersecurity specialist',
       'hr specialist', 'network engineer', 'graphic designer',
       'game developer', 'mobile app developer', 'cloud architect',
       'business analyst', 'database administrator', 'devops engineer',
       'machine learning engineer', 'content writer',
       'system administrator', 'ui/ux designer', 'ai engineer',
       'project manager', 'software developer'], dtype=object)

In [16]:
unique_count = data.groupby('Role')['ID'].count()
unique_count

Role
ai engineer                      13
business analyst                 11
cloud architect                   8
content writer                    5
cybersecurity specialist          9
data analyst                    267
data engineer                   509
data scientist                  592
database administrator            9
devops engineer                  16
digital marketing specialist     14
game developer                   10
graphic designer                 15
hr specialist                    12
machine learning engineer        13
mobile app developer              7
network engineer                 16
product manager                 514
project manager                  56
software developer               66
software engineer               551
system administrator              5
ui designer                     234
ui engineer                     291
ui/ux designer                    7
Name: ID, dtype: int64

In [17]:
## Dropping Rows which have count < 100 
for count , role in zip(unique_count.values, unique_count.index):
    if count < 100 :
        data = data[data['Role'] != role] 

In [18]:
unique_count = data.groupby('Role')['ID'].count()
unique_count

Role
data analyst         267
data engineer        509
data scientist       592
product manager      514
software engineer    551
ui designer          234
ui engineer          291
Name: ID, dtype: int64

In [19]:
data.shape

(2958, 8)

In [20]:
## Performing Feature Engineering

In [21]:
def textpreprocessing(text):
    sentences = re.split(r'(?<=[.!?])\s+', text)
    nlp = spacy.load("en_core_web_sm") 
    stopword_s = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()

    for idx, sent in enumerate(sentences):
        words = nlp(sent)

        words = [word.text for word in words if word.text.lower() not in stopword_s]
        words = [lemmatizer.lemmatize(word) for word in words]  
        
        sentences[idx] = ' '.join(words)  

    return ' '.join(sentences)  # Join the sentences back together

In [22]:
data['Transcript_processed'] = data['Transcript'].apply(textpreprocessing)
# Saving the processed Data to a temporary file
data.to_excel(path+'/'+'final_data.csv')

KeyboardInterrupt: 

In [ ]:
data['Resume_processed'] = data['Resume'].apply(textpreprocessing)
# Saving the processed Data to a temporary file
data.to_excel(path+'/'+'final_data.csv')

In [ ]:
data['Job_Description_processed'] = data['Job Description'].apply(textpreprocessing)
# Saving the processed Data to a temporary file
data.to_excel(path+'/'+'final_data.csv')